In [38]:
import pandas as pd
import numpy as np

In [39]:
pd.set_option('display.max_columns', None)

In [40]:
dns_laptop_df = pd.read_csv('data/dns_laptop_dataset_raw_new_column_names.csv')

## Объявим вспомогательные функции для обработки признаков

In [41]:
def get_backlight_type(text):
    if 'одноцветная' in text.lower():
        return 'single'
    if 'argb' in text.lower():
        return 'ARGB'
    if 'rgb' in text.lower():
        return 'RGB'
    return None

def get_screen_type(text):
    if 'tn' in text.lower():
        return 'TN'
    if 'ips' in text.lower():
        return 'IPS'
    if 'oled' in text.lower():
        return 'OLED'
    if 'mini-led' in text.lower():
        return 'mini-LED'
    return 'other'

def get_mc_slot_type(text):
    if text == 0:
        return None
    if 'microsd' in text.lower():
        return 'microSD'
    if 'sd' in text.lower():
        return 'SD'
    return None

# Функция для удаления единиц измерения
# return_rest_of - вместо числа вернуть то, что удалили
# fill_empty - чем заполнять пропуски
# skip_values - список значений, которые не обрабатываются
def delete_uom(series, f, return_rest_of=False, fill_empty=None, skip_values=None):
    
    # Первоначально заполним пропуски нулями
    series = series.fillna(0)
    
    # Если skip_values не передано, то создаем пустой список
    if skip_values is None:
        skip_values = []

    # Определим функции, которые используем в зависимость от return_rest_of
    clear_f = (lambda x:
        f(x.split()[0])
        if x != '0' and (x not in skip_values)
        else x
        )

    rest_f = (lambda x:
        ' '.join(x.split()[1:])
        if x != '0' and (x not in skip_values)
        else x
        )
    
    if return_rest_of:
        use_f = rest_f
    else:
        use_f = clear_f
    
    # Применяем выбраную функцию
    series = (
        series
        # .fillna(0)
        .astype('str')
        .apply(use_f)    
    )
    
    series = series.apply(lambda x: fill_empty if x == '0' else x)

    return series

## Трансформируем и создаем новые признаки

In [42]:
# Выделим размер скидки
dns_laptop_df['discount'] = dns_laptop_df['price'] - dns_laptop_df['price_discount']

# Рассчитаем процент скидки
dns_laptop_df['percentage_discount'] = (dns_laptop_df['discount'] / dns_laptop_df['price'] * 100).apply('round', args=(1,))

# Выделим срок гарантии в месяцах
dns_laptop_df['warranty'] = dns_laptop_df['warranty'].fillna(0)
dns_laptop_df['warranty'] = dns_laptop_df['warranty'].apply(lambda x: str(x).split()[0] if x != 0 else 0).astype('int')

# Определим тип как категориальный признак
dns_laptop_df['type'] = dns_laptop_df['type'].astype('category')

# Отделим брэнд
dns_laptop_df['brand'] = dns_laptop_df['model'].astype('str').apply(lambda x: x.split()[0])
dns_laptop_df['model_name'] = dns_laptop_df['model'].astype('str').apply(lambda x: ' '.join(x.split()[1:]))
dns_laptop_df = dns_laptop_df.drop('model', axis=1)

# Удалим квадратные скобки у model_id
dns_laptop_df['model_id'] = dns_laptop_df['model_id'].str[1:-1]

# Заполним пропуски в году нулем и приведем в int
dns_laptop_df['release_year'] = dns_laptop_df['release_year'].fillna(0).astype('int')

# Приведем game_notebook к булеву типу
dns_laptop_df['game_notebook'] = dns_laptop_df['game_notebook'].apply(lambda x: True if x == 'да' else False)

# Приведем form_factor к булеву типу
dns_laptop_df['transformer'] = dns_laptop_df['form_factor'].apply(lambda x: False if x == 'классический' else True)
dns_laptop_df = dns_laptop_df.drop('form_factor', axis=1)

# Выделим модели с русской раскладкой клавиатуры
dns_laptop_df['ru_keyboard'] = (
    dns_laptop_df['keyboard_layout']
    .astype('str')
    .apply(
        lambda x: True
        if 'русская' in x.lower()
        else False
        )
    )
dns_laptop_df = dns_laptop_df.drop('keyboard_layout', axis=1)

# Приведем numpad_layout к булеву типу
dns_laptop_df['numpad_layout'] = dns_laptop_df['numpad_layout'].apply(lambda x: True if x == 'да' else False)
dns_laptop_df = dns_laptop_df.drop('numpad_layout', axis=1)

# Выделим тип подсветки клавиатуры
dns_laptop_df['keyboard_backlight_type'] = (
    dns_laptop_df['keyboard_backlight'].astype('str')
    .apply(
        lambda x:
        get_backlight_type(x)
    )
    )

# Выделим модели с подсветкой клавиатуры
dns_laptop_df['keyboard_backlight'] = (
    dns_laptop_df['keyboard_backlight']
    .apply(
        lambda x: False
        if x == 'нет'
        else True
    )
    )

dns_laptop_df = dns_laptop_df.drop('keyboard_backlight', axis=1)

# Приведем fingerprint к булеву типу
dns_laptop_df['fingerprint'] = dns_laptop_df['fingerprint'].apply(lambda x: True if x == 'есть' else False)

# Приведем mech_keyboard к булеву типу
dns_laptop_df['mech_keyboard'] = dns_laptop_df['mech_keyboard'].apply(lambda x: True if x == 'есть' else False)

# Выделим тип матрицы дисплея
dns_laptop_df['screen_type'] = (
    dns_laptop_df['screen_type'].astype('str')
    .apply(
        lambda x:
        get_screen_type(x)
    )
    )

# Выделим размер дисплея
dns_laptop_df['screen_size'] = (
    dns_laptop_df['screen_size'].astype('str')
    .apply(
        lambda x:
        float(x.replace('\"', ''))
    )
    )

# Выделим частоту дисплея
dns_laptop_df['screen_frec'] = (
    dns_laptop_df['screen_frec']
    .fillna(0)
    .astype('str')
    .apply(
        lambda x:
        int(x.replace('Гц', ''))
        if x != '0'
        else None
    )
    )

# Выделим яркость дисплея
dns_laptop_df['screen_bright'] = (
    dns_laptop_df['screen_bright']
    .fillna(0)
    .astype('str')
    .apply(
        lambda x:
        int(x.split()[0])
        if x != '0'
        else None
    )
    )

# Выделим ppi дисплея
dns_laptop_df['screen_ppi'] = delete_uom(dns_laptop_df['screen_ppi'], float)

# Приведем fingerprint к булеву типу
dns_laptop_df['screen_sensor'] = dns_laptop_df['screen_sensor'].apply(lambda x: True if x == 'есть' else False)

# Выделим производитела процессора
dns_laptop_df['cpu_vendor'] = delete_uom(dns_laptop_df['cpu_model'], str)

# Выделим модель процессора
dns_laptop_df['cpu_model'] = delete_uom(dns_laptop_df['cpu_model'], str, return_rest_of=True)

# Обработаем cpu_core_effective
dns_laptop_df['cpu_core_effective'] = (
    dns_laptop_df['cpu_core_effective']
    .fillna(0)
    .apply(
        lambda x:
        int(x)
        if x != 'нет'
        else 0
    )
    )      

# Заменим пропуск в количестве ядер на сумму productive + effective
dns_laptop_df['cpu_core'] = dns_laptop_df['cpu_core'].fillna(0)
(
    dns_laptop_df
    .apply(
        lambda x: x['cpu_core_productive'] + x['cpu_core_effective']
        if x['cpu_core'] == 0
        else x['cpu_core'],
        axis=1
        )
    )

# Выделим частоту процессора
dns_laptop_df['cpu_freq'] = delete_uom(dns_laptop_df['cpu_freq'], float, fill_empty=0)

# Выделим частоту процессора в разгоне
dns_laptop_df['cpu_accelerated'] = (
    dns_laptop_df['cpu_accelerated']
    .fillna('0 ГГц')
    .apply(
        lambda x:
        float(x.split()[0])
        if x != 'нет'
        else 0
    )
    )  

# Если нет частоты в разгоне, то заполим базовой
dns_laptop_df['cpu_accelerated']  = dns_laptop_df.apply(
        lambda x: x['cpu_freq']
        if x['cpu_accelerated'] == 0
        else x['cpu_accelerated'],
        axis=1
        )

# Выделим объём ОЗУ
dns_laptop_df['ram_capacity'] = delete_uom(dns_laptop_df['ram_capacity'], int)

# Выделим частоту ОЗУ
dns_laptop_df['ram_freq'] = delete_uom(dns_laptop_df['ram_freq'], int)

# Выделим тип GPU
dns_laptop_df['discrete_gpu'] = (
    dns_laptop_df['gpu_type']
    .apply(
        lambda x:
        True
        if 'дискретный' in x
        else False
    )
)
dns_laptop_df = dns_laptop_df.drop('gpu_type', axis=1)

# Отделим производителя дискретного GPU
dns_laptop_df['gpu_vendor_disc'] = delete_uom(dns_laptop_df['gpu_model_disc'], str)
# Заполним пропуски
dns_laptop_df['gpu_vendor_disc'] = (
    dns_laptop_df['gpu_vendor_disc'].
    apply(
        lambda x:
        None
        if x == 'нет'
        else x
    )
)

# Выделим модель дискретного GPU
dns_laptop_df['gpu_model_disc'] = delete_uom(dns_laptop_df['gpu_model_disc'], str, return_rest_of=True)
dns_laptop_df['gpu_model_disc'] = dns_laptop_df['gpu_model_disc'].apply(lambda x: None if x == '' else x)

# Выделим объём SSD
dns_laptop_df['ssd_volume'] = delete_uom(dns_laptop_df['ssd_volume'], int, fill_empty=0, skip_values=['нет'])
dns_laptop_df['ssd_volume'] = dns_laptop_df['ssd_volume'].apply(lambda x: 0 if x == 'нет' else x)

# Выделим объём HDD
dns_laptop_df['hdd_volume'] = delete_uom(dns_laptop_df['hdd_volume'], int, fill_empty=0, skip_values=['нет'])
dns_laptop_df['hdd_volume'] = dns_laptop_df['hdd_volume'].apply(lambda x: 0 if x == 'нет' else x)

# Выделим объём eMMC
dns_laptop_df['emmc_volume'] = delete_uom(dns_laptop_df['emmc_volume'], int, fill_empty=0, skip_values=['нет'])
dns_laptop_df['emmc_volume'] = dns_laptop_df['emmc_volume'].apply(lambda x: 0 if x == 'нет' else x)

# Выделим число число мегапикселей вебкамеры
dns_laptop_df['webcam'] = delete_uom(dns_laptop_df['webcam'], float, skip_values=['нет'])
dns_laptop_df['webcam'] = dns_laptop_df['webcam'].apply(lambda x: 0 if x == 'нет' else x)



# Определим поддержку MMC
dns_laptop_df['card_type'] = dns_laptop_df['card_type'].fillna(0)
dns_laptop_df['card_slot_mmc_support'] = (
    dns_laptop_df['card_type']
    .apply(
        lambda x:
        True
        if 'mmc' in str(x).lower()
        else False
    )
)

# Выделим тип слота под карту памяти
dns_laptop_df['card_slot_type'] = (
    dns_laptop_df['card_type']
    .apply(get_mc_slot_type)
)


# Выделим отдельно wi-fi, bluetooth и cellular
dns_laptop_df[['wifi', 'bluetooth', 'cellular']] = dns_laptop_df.wireless_interface.str.split(',', 2, expand=True)
dns_laptop_df = dns_laptop_df.drop('wireless_interface', axis=1)

# Выделим наличие ethernet порта

dns_laptop_df['ethernet_port_speed'] = (
    dns_laptop_df['ethernet_port']
    .apply(
        lambda x: float(x.split()[1])
        if x != 'нет'
        else None
    )
)
dns_laptop_df['ethernet_port'] = (
    dns_laptop_df['ethernet_port']
    .apply(
        lambda x:
        False
        if x == 'нет'
        else True
    )
)

In [43]:
# Приведем usb_power_delivery к булеву типу
dns_laptop_df['usb_power_delivery'] = dns_laptop_df['usb_power_delivery'].apply(lambda x: True if x == 'есть' else False)

# Выделим емкость аккумулятора
dns_laptop_df['acc_time'] = dns_laptop_df['acc_time'].astype('str').apply(lambda x: x.split()[0]).astype('float')

# Выделим геометрические размеры
dns_laptop_df['depth'] = dns_laptop_df['depth'].astype('str').apply(lambda x: x.split()[0]).astype('float')
dns_laptop_df['width'] = dns_laptop_df['width'].astype('str').apply(lambda x: x.split()[0]).astype('float')
dns_laptop_df['height'] = dns_laptop_df['height'].astype('str').apply(lambda x: x.split()[0]).astype('float')
dns_laptop_df['weight'] = dns_laptop_df['weight'].astype('str').apply(lambda x: x.split()[0]).astype('float')

# Выделим максимальный объём ОЗУ
dns_laptop_df['ram_max_volume'] = dns_laptop_df['ram_max_volume'].astype('str').apply(lambda x: x.split()[0])
dns_laptop_df.loc[(dns_laptop_df.ram_max_volume == 'nan'), 'ram_max_volume'] = np.nan

# Выделим тип SSD
dns_laptop_df['ssd_type'] = dns_laptop_df['ssd_type'].astype('str').apply(lambda x: x.split()[1] if len(x.split())>1 else None)

# Выделим мощность адаптера питания
dns_laptop_df['adapter_output_power'] = dns_laptop_df['adapter_output_power'].astype('str').apply(lambda x: x.split()[0])
dns_laptop_df.loc[(dns_laptop_df.adapter_output_power == 'nan'), 'adapter_output_power'] = np.nan

# Выделим размер ОЗУ видеокарты
dns_laptop_df['gddr_volume'] = dns_laptop_df['gddr_volume'].astype('str').apply(lambda x: x.split()[0])
dns_laptop_df.loc[(dns_laptop_df.gddr_volume == 'nan'), 'gddr_volume'] = np.nan

# Выделим значение тактовой частоты процессора в режиме аккселерации
dns_laptop_df['freq_with_acc'] = dns_laptop_df['freq_with_acc'].astype('str').apply(lambda x: x.split()[0])
dns_laptop_df.loc[(dns_laptop_df.freq_with_acc == 'nan'), 'freq_with_acc'] = np.nan

# Выделим максимальной потребляемой мощности
dns_laptop_df['power_consumption_max'] = dns_laptop_df['power_consumption_max'].astype('str').apply(lambda x: x.split()[0])
dns_laptop_df.loc[(dns_laptop_df.power_consumption_max == 'nan'), 'power_consumption_max'] = np.nan

## Удаляем ненужные признаки

In [44]:
dns_laptop_df = dns_laptop_df.drop('mic', axis=1)
dns_laptop_df = dns_laptop_df.drop('optical_drive', axis=1)

dns_laptop_df.drop('features', axis=1, inplace=True)
dns_laptop_df.drop('specification', axis=1, inplace=True)
dns_laptop_df.drop('advanced_features', axis=1, inplace=True)
# Признак color_manufacturer предлагаю дропнуть - цвет уже был указан в другом признаке, а этот ко всему прочему имеет кучу пропусков
dns_laptop_df.drop('color_manufacturer', axis=1, inplace=True)
# Признак скорости вращения шпинделя HDD предлагаю дропнуть
dns_laptop_df.drop('hdd_spin_vel', axis=1, inplace=True)
# Страна-производитель указана для очень ограниченного числа ноутбуков и только для Китая, поэтому убираем эту колонку
dns_laptop_df.drop('country_of_manufacturer', axis=1, inplace=True)


In [45]:
dns_laptop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714 entries, 0 to 713
Data columns (total 90 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   rating                   714 non-null    float64 
 1   votes                    714 non-null    int64   
 2   service_rating           714 non-null    float64 
 3   price                    714 non-null    int64   
 4   price_discount           714 non-null    int64   
 5   warranty                 714 non-null    int64   
 6   type                     714 non-null    category
 7   model_id                 693 non-null    object  
 8   release_year             714 non-null    int64   
 9   operating_system         714 non-null    object  
 10  game_notebook            714 non-null    bool    
 11  cover_color              714 non-null    object  
 12  cover_material           712 non-null    object  
 13  body_material            714 non-null    object  
 14  touchpad  

In [46]:
dns_laptop_df

,rating,votes,service_rating,price,price_discount,warranty,type,model_id,release_year,operating_system,game_notebook,cover_color,cover_material,body_material,touchpad,fingerprint,mech_keyboard,screen_type,screen_size,screen_res,screen_coat,screen_sensor,screen_frec,screen_bright,screen_ppi,screen_sync,cpu_model,cpu_core,cpu_core_productive,cpu_core_effective,cpu_thread,cpu_freq,cpu_accelerated,ram_type,ram_capacity,ram_freq,gpu_model_int,gpu_model_disc,ssd_volume,hdd_volume,emmc_volume,webcam,card_type,ethernet_port,video_port,audio_port,usb_type_a,usb_type_c,additional_interfaces,thunderbolt,usb_power_delivery,acc_type,acc_capacity,acc_time,depth,width,height,weight,ram_slots,ram_max_volume,ram_free_slot,second_display,video_port_type,tpm_support,ssd_type,seller_warranty,screen_color_depth,adapter_output_power,gpu_manufacturer,video_type,gddr_volume,freq_with_acc,screen_hdr,power_consumption_max,discount,percentage_discount,brand,model_name,transformer,ru_keyboard,keyboard_backlight_type,cpu_vendor,discrete_gpu,gpu_vendor_disc,card_slot_mmc_support,card_slot_type,wifi,bluetooth,cellular,ethernet_port_speed
0,3.5,89,93.11,14999,14999,12,ноутбук,NB77,2021,Windows 10 Home Single Language,False,черный,пластик,пластик,стандартный тачпад,False,False,TN,13.3,HD (1366x768),матовое,False,60.0,200.0,118.0,нет,Atom Z3735F,4.0,4,0,4,1.33,1.83,DDR3,2,1333.0,Intel HD Graphics,None,0,0,32,0.3,нет,False,mini HDMI,3.5 мм jack (микрофон/аудио),USB 2.0 x2,нет,нет,нет,False,Li-Ion,8000 мА*ч,5.0,217.5,329.3,20.8,1.300,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Irbis,NB77,False,True,None,Intel,False,None,False,None,WI-FI 4 (802.11n),Bluetooth 4.0,None,NaN
1,4.5,52,97.67,15999,15999,12,ноутбук,NB257,2021,Windows 10 Home Single Language,False,серый,пластик,пластик,стандартный тачпад,False,False,IPS,14.0,HD (1366x768),матовое,False,60.0,180.0,112.0,нет,Celeron 3350,2.0,2,0,2,1.10,2.40,LPDDR4,4,NaN,Intel HD Graphics 500,None,0,0,64,0.3,0,False,mini HDMI,3.5 мм jack (микрофон/аудио),USB 2.0,USB 3.2 Gen1,нет,нет,False,Li-Ion,4500 мА*ч,3.5,217.5,329.3,20.8,1.273,интегрированная,4,нет,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Irbis,NB257,False,True,None,Intel,False,None,False,None,WI-FI 4 (802.11n),Bluetooth 4.0,None,NaN
2,4.0,27,98.70,15999,15999,12,ноутбук,NB270,2021,Windows 10 Pro,False,белый,пластик,пластик,стандартный тачпад,False,False,IPS,14.1,Full HD (1920x1080),матовое,False,60.0,200.0,156.0,нет,Celeron N4020,2.0,2,0,2,1.10,2.80,LPDDR4,4,2400.0,Intel UHD Graphics 600,None,0,0,128,2.0,microSD,False,mini HDMI,3.5 мм jack (микрофон/аудио),"USB 2.0, USB 3.2 Gen1",нет,NaN,нет,False,Li-Ion,4000 мА*ч,NaN,213.0,328.0,21.0,1.280,интегрированная,4,нет,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Irbis,NB270,False,True,None,Intel,False,None,False,microSD,WI-FI 4 (802.11n),Bluetooth 4.0,None,NaN
3,2.5,3,97.76,17999,17999,12,ноутбук,ES4060EW,2022,Windows 10 Home Single Language,False,серый,пластик,пластик,стандартный тачпад,False,False,IPS,14.1,Full HD (1920x1080),NaN,False,60.0,NaN,155.0,нет,Celeron N4020,2.0,2,0,2,1.10,2.80,DDR4,4,NaN,Intel UHD Graphics 600,None,0,0,64,0.3,microSD,False,mini HDMI,3.5 мм jack (микрофон/аудио),"USB 2.0, USB 3.2 Gen1",нет,нет,нет,False,Li-Ion,5000 мА*ч,5.3,216.0,327.0,23.0,1.250,NaN,NaN,NaN,нет,HDMI 1.4,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Digma,EVE 14 C414,False,True,None,Intel,False,None,False,microSD,WI-FI 4 (802.11n),Bluetooth 4.0,None,NaN
4,0.0,0,0.00,19199,19199,12,ноутбук,NB283,2021,Windows 10 Pro,False,серый,пластик,пластик,стандартный тачпад,False,False,IPS,14.0,HD (1366x768),матовое,False,60.0,180.0,112.0,нет,Celeron 3350,2.0,2,0,2,1.10,2.40,LPDDR4,4,NaN,Intel HD Graphics 500,None,0,0,128,0.3,microSD,False,mini HDMI,3.5 мм jack (микрофон/аудио),"USB 2.0, USB 3.2 Gen1",нет,нет,нет,False,Li-Ion,4500 мА*ч,3.5,217.5,329.3,20.8,1.273,интегрированная,4,нет,нет,HDMI 1.4,нет,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Irbis,NB283,False,True,None,Inte

## Выгружаем очищенный датасет

In [47]:
dns_laptop_df.to_csv('data/dns_laptop_dataset.csv', index=False)